In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
from sympy import *
import plotly.offline as py
import plotly.graph_objs as go
%matplotlib inline
init_printing()

In [10]:
Variables=symbols("x1 y1 x2 y2")
x1,y1,x2,y2 = Variables
Time=symbols("t")
t=Time
Parameters = symbols("a b alpha beta gamma delta w_a w_b a0 a1 omega epsilon")
a,b,alpha,beta,gamma,delta,w_a,w_b,a0,a1,omega,epsilon = Parameters

### duffing system

In [11]:
F=Matrix([y1, gamma*cos(omega*t) - delta*y1 - beta*x1 - alpha*x1**3,
          y2, gamma*cos(omega*t) - delta*y2 - beta*x2 - alpha*x2**3])
F

⎡                y₁                ⎤
⎢                                  ⎥
⎢      3                           ⎥
⎢- α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(ω⋅t)⎥
⎢                                  ⎥
⎢                y₂                ⎥
⎢                                  ⎥
⎢      3                           ⎥
⎣- α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(ω⋅t)⎦

In [12]:
zeta=Matrix([0,0,
             0,0])
zeta

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

### synchrony

In [13]:
phi=Matrix([(x2-x1*(a*sin(w_a*t)+b*sin(w_b*t)))])
phi

[-x₁⋅(a⋅sin(t⋅wₐ) + b⋅sin(t⋅w_b)) + x₂]

In [14]:
N=simulation.VectorDivergence(phi,Variables)
N

[-a⋅sin(t⋅wₐ) - b⋅sin(t⋅w_b)  0  1  0]

In [15]:
Ndot = diff(N,Time)
Ndot

[-a⋅wₐ⋅cos(t⋅wₐ) - b⋅w_b⋅cos(t⋅w_b)  0  0  0]

In [16]:
on_manifold = [(x2,x1*(a*sin(w_a*t)+b*sin(w_b*t)))]

### invariance

In [17]:
NF=N*(F+zeta)
NF

[y₁⋅(-a⋅sin(t⋅wₐ) - b⋅sin(t⋅w_b)) + y₂]

In [18]:
dPhi=diff(phi,Time)
dPhi

[-x₁⋅(a⋅wₐ⋅cos(t⋅wₐ) + b⋅w_b⋅cos(t⋅w_b))]

In [19]:
eqn=(NF+dPhi)
eqn

[-x₁⋅(a⋅wₐ⋅cos(t⋅wₐ) + b⋅w_b⋅cos(t⋅w_b)) + y₁⋅(-a⋅sin(t⋅wₐ) - b⋅sin(t⋅w_b)) + 
y₂]

In [20]:
zeta[2]=-(eqn)[0]

In [21]:
zeta=zeta.subs(on_manifold)
zeta

⎡                                      0                                      
⎢                                                                             
⎢                                      0                                      
⎢                                                                             
⎢x₁⋅(a⋅wₐ⋅cos(t⋅wₐ) + b⋅w_b⋅cos(t⋅w_b)) - y₁⋅(-a⋅sin(t⋅wₐ) - b⋅sin(t⋅w_b)) - y
⎢                                                                             
⎣                                      0                                      

 ⎤
 ⎥
 ⎥
 ⎥
₂⎥
 ⎥
 ⎦

In [22]:
NF=N*(F+zeta)
simplify(NF)
simplify(NF+dPhi).subs(on_manifold)

[0]

In [23]:
NZ=N*zeta
NZ

[x₁⋅(a⋅wₐ⋅cos(t⋅wₐ) + b⋅w_b⋅cos(t⋅w_b)) - y₁⋅(-a⋅sin(t⋅wₐ) - b⋅sin(t⋅w_b)) - y
₂]

### stability

In [26]:
chi=Matrix([0,0,-1,0])*phi

chi

⎡                 0                  ⎤
⎢                                    ⎥
⎢                 0                  ⎥
⎢                                    ⎥
⎢x₁⋅(a⋅sin(t⋅wₐ) + b⋅sin(t⋅w_b)) - x₂⎥
⎢                                    ⎥
⎣                 0                  ⎦

In [27]:
J=simplify((Ndot+simulation.VectorDivergence(N*(F+zeta+chi),Variables))*N.transpose())
J

⎡                              2    ⎤
⎣- (a⋅sin(t⋅wₐ) + b⋅sin(t⋅w_b))  - 1⎦